In [9]:
import numpy as np
import pandas as pd
from pandas.tseries.offsets import *
import json
import logging

In [10]:
today_date = pd.to_datetime('today').date()
logging.basicConfig(filename=f'logs/{today_date}.log', level=logging.INFO)
logger = logging.getLogger(__name__)

In [11]:
class json_basic:
    def __init__(self) -> None:
        pass
        
    def readJson(self, file_name, folder_name:str = "")->dict:
        '''Returns a dictionary of the information in JSON file. If file doesn't exist, then return a None value.'''
        if(type(file_name) == pd.Timestamp):
            file_name = file_name.date()
        
        if(folder_name == ""):
            try:
                f = open(f"{file_name}.json")
                data = json.load(f)
                f.close
                return data
            except:
                return None
            
        try:
            f = open(f"{folder_name}/{file_name}.json")
            data = json.load(f)
            f.close
            return data
        except:
            return None
        
    def writeJson(self, dt:dict, file_name, folder_name:str = "")->None:
        '''Writes the dictionary in the JSON file, with given file name and folder name.'''
        if(type(file_name) == pd.Timestamp):
            file_name = file_name.date()
        
        if(folder_name == ""):
            json_object = json.dumps(dt, indent=4)
            with open(f"indexes/{file_name}.json", "w") as outfile:
                outfile.write(json_object)
        else:
            json_object = json.dumps(dt, indent=4)
            with open(f"{folder_name}/{file_name}.json", "w") as outfile:
                outfile.write(json_object)

jb = json_basic()

In [19]:
data = jb.readJson("parameters5risk")
#converting dates from str datatype to datetime datatype
for key in data:
    if key[-4:] == "date":
        data[key] = pd.to_datetime(data[key])

ipn = data['index_prcsn']
calculation_days = pd.bdate_range(data['st_date'], data['end_date'])

In [21]:
df = pd.read_excel(data['underlying_file_path'], sheet_name=None, index_col=0)

In [22]:
all_dates = df['GLD'].index.to_list()

#### Class to calculate 5% risk control index

In [24]:
class index5risk(json_basic):
    def __init__(self, inc_date: pd.Timestamp, name:str, assets:str) -> None:
        '''Parameters:
            inception_date: Inception Date of the index
            Assets: List of assets of respective index
            name: Name of the index'''    
        self.inc_date = inc_date
        self.name = name
        self.assets = assets

    def il(self, dt:dict)->float:
        '''Returns IL from the given dict, 0 if dict is empty\n
        Parameters:
            dt: dictionary containing today's portfolio of respective index only'''
        if not dt:
            return 0
        if self.name not in dt.keys():
            return 0
        return dt[self.name]['IL']

    def uil(self, dt:dict)->float:
        '''Returns UIL from the given dict, 0 if dict is empty\n
        Parameters:
            dt: dictionary containing today's portfolio of respective index only'''
        if not dt:
            return 0
        return dt[self.assets]['IL']

    def units(self, dt:dict)->float:
        '''Returns units from the given dict, 0 if dict is empty\n
        Parameters:
            dt: dictionary containing today's portfolio of respective index only'''
        if not dt:
            return 0
        if self.name not in dt.keys():
            return 0
        return dt[self.name]['UNITS']

    def uivol(self, dt:dict)->float:
        '''Returns UIVol from the given dict, 0 if dict is empty\n
        Parameters:
            dt: dictionary containing today's portfolio of respective index only'''
        if not dt:
            return 0
        if self.name not in dt.keys():
            return 0
        return dt[self.name]['UIVol']

    def uiw(self, dt:dict)->float:
        '''Returns UIW from the given dict, 0 if dict is empty\n
        Parameters:
            dt: dictionary containing today's portfolio of respective index only'''
        if not dt:
            return 0
        if self.name not in dt.keys():
            return 0
        return dt[self.name]['UIW']


    def next_date(self, date: pd.Timestamp, n:int) -> pd.Timestamp:
        '''Returns a date "n" buisness days after the given date\n
        Parameters:
            date: beginning date for calculation
            n: number of buisness days'''
        if date in all_dates:
            i = all_dates.index(date)
            if(i+n < len(all_dates) and i+n >= 0):
                return all_dates[i+n]
        return date + BusinessDay(n)

    def calcSqrLogReturn(self, st_date: pd.Timestamp, end_date: pd.Timestamp)->float:
        '''Returns summation of square of log returns of daily portfolios, from st_date to end_date (both inclusive)\n
        Parameters:
            st_date: First date for calculation (Inclusive)
            end_date: Last date for calculation (Inclusive)'''
        res = 0
        prev_date = self.next_date(st_date, -1)
        prev_date_data = self.readJson(prev_date, data['folder_name'])
        pdd_uil = self.uil(prev_date_data)
        if(pdd_uil == 0):
            return 0
        
        date = st_date
        while date <= end_date:
            date_data = self.readJson(date, data['folder_name'])
            date_uil = self.uil(date_data)
            res += (np.log(date_uil/pdd_uil)**2)
            pdd_uil = date_uil
            date = self.next_date(date, 1)

        return res

    def calcUiVolLt(self, date: pd.Timestamp)->float:
        '''It return underlying index's volatility for long term(60 days)\n
        Parameters:
            date: date of calculation day'''
        sum = self.calcSqrLogReturn(self.next_date(date, 1 - data['LT']), date)
        res = np.sqrt(252*sum/data['LT'])
        logger.info(f"Long term UI Volatility is {res} for date : {date}")
        return res
    
    def calcUiVolSt(self, date: pd.Timestamp)->float:
        '''It return underlying index's volatility for long term(20 days)\n
        Parameters:
            date: date of calculation day'''
        sum = self.calcSqrLogReturn(self.next_date(date, 1 - data['ST']), date)
        res = np.sqrt(252*sum/data['ST'])
        logger.info(f"Short term UI Volatility is {res} for date : {date}")
        return res

    def calcUiVol(self, date: pd.Timestamp)->float:
        '''It return underlying index's volatility which will be used for calculation\n
        Parameters:
            date: date of calculation day'''
        res = max(self.calcUiVolLt(date), self.calcUiVolSt(date))
        logger.info(f"UI Volatility is {res} for date : {date}")
        return res
    
    def calcUiw(self, date: pd.Timestamp, pdd:dict, dt:dict)->float:
        '''It returns the underlying index's weight\n
        Parameters:
            date: date of calculation day
            dt: dictionary containing today's portfolio of respective index only
            pdd: dictionary containing portfolio of last calculation day'''
        res = None
        if(date == self.inc_date or dt['R_DAY']):
            res = min(data['Cap'], data['VT']/self.uivol(pdd))
        else:
            res = self.uiw(pdd)

        logger.info(f"UI Weight is {res} for date : {date}")
        return res
    
    def calcUnits(self, date: pd.Timestamp, pdd:dict, dt:dict)->float:
        '''It returns the units for underlying index\n
        Parameters:
            date: date of calculation day
            dt: dictionary containing today's portfolio of respective index only
            pdd: dictionary containing portfolio of last calculation day'''
        res = None
        if(date == self.inc_date):
            res = dt['UIW']*1000/dt['UIL']
        elif(dt['R_DAY']):
            res = dt['UIW']*self.il(pdd)/self.uil(pdd)
        else:
            res = self.units(pdd)

        logger.info(f"Units are {res} for date : {date}")
        return res

    def calcRday(self, date: pd.Timestamp, pdd:dict)->bool:
        '''Returns whether a calculation day will be a rebalance day or not\n
        Parameters:
            date: date of calculation day
            pdd: dictionary containing portfolio of last calculation day'''
        rday = False
        if(date == self.inc_date):
            rday = True
        elif(date > self.inc_date):
            pd_uiw = self.uiw(pdd)
            pd_uivol = self.uivol(pdd)
            if(pd_uivol != 0 and pd_uiw != 0):
                uiw_temp = min(data['Cap'], data['VT']/pd_uivol)
                if(abs(uiw_temp - pd_uiw)/pd_uiw >= data['R_Threshold']):
                    rday = True
        logger.info(f"Date:{date} is marked {rday} as rebalance day for {self.name} index")
        return rday

    def calcTc(self, date: pd.Timestamp, pdd:dict, dt:dict)->float:
        '''It returns the Transaction cost of the calculation day\n
        Parameters:
            date: date of calculation day
            dt: dictionary containing today's portfolio of respective index only
            pdd: dictionary containing portfolio of last calculation day'''
        res = abs(dt['UNITS'] - self.units(pdd))*dt['UIL']*data['TC_RATE']
        logger.info(f"Transaction cost is {res} for date : {date}")
        return res

    def calcFee(self, date: pd.Timestamp, pdd:dict) -> float:
        '''It returns index's fees for the calculation day\n
        Parameters:
            date: date of calculation day
            pdd: dictionary containing portfolio of last calculation day'''
        prev_date = self.next_date(date, -1)
        res = self.il(pdd)*data['FEE_RATE']*(date - prev_date).days/360
        logger.info(f"Index's fees is {res} for date : {date}")
        return res
    
    def calcIl(self, date: pd.Timestamp, pdd:dict, dt:dict) -> float:
        '''It calculates the index for the calculation day\n
        Parameters:
            date: date of calculation day
            dt: dictionary containing today's portfolio of respective index only
            pdd: dictionary containing portfolio of last calculation day'''
        if(date == self.inc_date):
            return 1000

        il = self.il(pdd)
        il += self.units(pdd)*(dt['UIL'] - self.uil(pdd))
        il -= dt['TC']
        il -= dt['FEE']
        il = round(il, ipn)
        logger.info(f"Index level is {il} for date : {date}")
        return il

    def calculate(self, date: pd.Timestamp, todays:dict)->dict:
        '''This function takes date, and todays(portfolio data) and calculate the portfolio for the calculation day and returns it through a dictionary\n
        Parameters:
            date: date of calculation day
            todays: dictionary containing today's information of index's assets'''
        next_date = self.next_date(date, 1)
        if(next_date < self.inc_date):
            logger.warning(f"{self.name} index doesn't exist for date: {date}")
            return todays
        dt = {}
        prev_date = self.next_date(date, -1)
        prev_date_data = self.readJson(prev_date, data['folder_name'])
        logger.info(f"Started working on portfolio on date : {date}")
        dt['UIL'] = todays[self.assets]['IL']
        dt['UIVol'] = self.calcUiVol(date)
        dt['R_DAY'] = self.calcRday(date, prev_date_data)
        dt['UIW'] = self.calcUiw(date, prev_date_data, dt)
        dt['UNITS'] = self.calcUnits(date, prev_date_data, dt)
        dt['TC'] = self.calcTc(date, prev_date_data, dt)
        dt['FEE'] = self.calcFee(date, prev_date_data)
        dt['IL'] = self.calcIl(date, prev_date_data, dt)
        logger.info(f"Finished working on portfolio on date : {date}")

        todays[self.name] = dt
        return todays
    
i5 = index5risk(pd.to_datetime('2007-03-30'), "5risk", 'RBC')

<style>
div.warn {    
    background-color: #dfb5b4;
    border: 5px solid #dfb5b4;
    padding: 0.5em;
    }
 </style>
<div class="warn">
    Warning: Below code will change the JSON files
</div>

#### Updating JSON files with index calculation

In [25]:
for date in calculation_days:
    if date not in all_dates:
        continue
    todays = jb.readJson(date, data['folder_name'])
    today_v4 = i5.calculate(date, todays)
    jb.writeJson(today_v4, date, data['folder_name'])